In [64]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding , Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import warnings
warnings.filterwarnings('ignore')

In [3]:
data_text = """The unanimous Declaration of the thirteen united States of America, When in the Course of human events, it becomes necessary for one people to dissolve the political bands which have connected them with another, and to assume among the powers of the earth, the separate and equal station to which the Laws of Nature and of Nature's God entitle them, a decent respect to the opinions of mankind requires that they should declare the causes which impel them to the separation.
We hold these truths to be self-evident, that all men are created equal, that they are endowed by their Creator with certain unalienable Rights, that among these are Life, Liberty and the pursuit of Happiness.--That to secure these rights, Governments are instituted among Men, deriving their just powers from the consent of the governed, --That whenever any Form of Government becomes destructive of these ends, it is the Right of the People to alter or to abolish it, and to institute new Government, laying its foundation on such principles and organizing its powers in such form, as to them shall seem most likely to effect their Safety and Happiness. Prudence, indeed, will dictate that Governments long established should not be changed for light and transient causes; and accordingly all experience hath shewn, that mankind are more disposed to suffer, while evils are sufferable, than to right themselves by abolishing the forms to which they are accustomed. But when a long train of abuses and usurpations, pursuing invariably the same Object evinces a design to reduce them under absolute Despotism, it is their right, it is their duty, to throw off such Government, and to provide new Guards for their future security.--Such has been the patient sufferance of these Colonies; and such is now the necessity which constrains them to alter their former Systems of Government. The history of the present King of Great Britain is a history of repeated injuries and usurpations, all having in direct object the establishment of an absolute Tyranny over these States. To prove this, let Facts be submitted to a candid world.
He has refused his Assent to Laws, the most wholesome and necessary for the public good.
He has forbidden his Governors to pass Laws of immediate and pressing importance, unless suspended in their operation till his Assent should be obtained; and when so suspended, he has utterly neglected to attend to them.
He has refused to pass other Laws for the accommodation of large districts of people, unless those people would relinquish the right of Representation in the Legislature, a right inestimable to them and formidable to tyrants only.
He has called together legislative bodies at places unusual, uncomfortable, and distant from the depository of their public Records, for the sole purpose of fatiguing them into compliance with his measures.
He has dissolved Representative Houses repeatedly, for opposing with manly firmness his invasions on the rights of the people.
He has refused for a long time, after such dissolutions, to cause others to be elected; whereby the Legislative powers, incapable of Annihilation, have returned to the People at large for their exercise; the State remaining in the mean time exposed to all the dangers of invasion from without, and convulsions within.
He has endeavoured to prevent the population of these States; for that purpose obstructing the Laws for Naturalization of Foreigners; refusing to pass others to encourage their migrations hither, and raising the conditions of new Appropriations of Lands.
He has obstructed the Administration of Justice, by refusing his Assent to Laws for establishing Judiciary powers.
He has made Judges dependent on his Will alone, for the tenure of their offices, and the amount and payment of their salaries.
He has erected a multitude of New Offices, and sent hither swarms of Officers to harrass our people, and eat out their substance.
He has kept among us, in times of peace, Standing Armies without the Consent of our legislatures.
He has affected to render the Military independent of and superior to the Civil power.
He has combined with others to subject us to a jurisdiction foreign to our constitution, and unacknowledged by our laws; giving his Assent to their Acts of pretended Legislation:
For Quartering large bodies of armed troops among us:
For protecting them, by a mock Trial, from punishment for any Murders which they should commit on the Inhabitants of these States:
For cutting off our Trade with all parts of the world:
For imposing Taxes on us without our Consent:
For depriving us in many cases, of the benefits of Trial by Jury:
For transporting us beyond Seas to be tried for pretended offences
For abolishing the free System of English Laws in a neighbouring Province, establishing therein an Arbitrary government, and enlarging its Boundaries so as to render it at once an example and fit instrument for introducing the same absolute rule into these Colonies:
For taking away our Charters, abolishing our most valuable Laws, and altering fundamentally the Forms of our Governments:
For suspending our own Legislatures, and declaring themselves invested with power to legislate for us in all cases whatsoever.
He has abdicated Government here, by declaring us out of his Protection and waging War against us.
He has plundered our seas, ravaged our Coasts, burnt our towns, and destroyed the lives of our people.
He is at this time transporting large Armies of foreign Mercenaries to compleat the works of death, desolation and tyranny, already begun with circumstances of Cruelty & perfidy scarcely paralleled in the most barbarous ages, and totally unworthy the Head of a civilized nation.
He has constrained our fellow Citizens taken Captive on the high Seas to bear Arms against their Country, to become the executioners of their friends and Brethren, or to fall themselves by their Hands.
He has excited domestic insurrections amongst us, and has endeavoured to bring on the inhabitants of our frontiers, the merciless Indian Savages, whose known rule of warfare, is an undistinguished destruction of all ages, sexes and conditions.
In every stage of these Oppressions We have Petitioned for Redress in the most humble terms: Our repeated Petitions have been answered only by repeated injury. A Prince whose character is thus marked by every act which may define a Tyrant, is unfit to be the ruler of a free people.
Nor have We been wanting in attentions to our Brittish brethren. We have warned them from time to time of attempts by their legislature to extend an unwarrantable jurisdiction over us. We have reminded them of the circumstances of our emigration and settlement here. We have appealed to their native justice and magnanimity, and we have conjured them by the ties of our common kindred to disavow these usurpations, which, would inevitably interrupt our connections and correspondence. They too have been deaf to the voice of justice and of consanguinity. We must, therefore, acquiesce in the necessity, which denounces our Separation, and hold them, as we hold the rest of mankind, Enemies in War, in Peace Friends.
We, therefore, the Representatives of the united States of America, in General Congress, Assembled, appealing to the Supreme Judge of the world for the rectitude of our intentions, do, in the Name, and by Authority of the good People of these Colonies, solemnly publish and declare, That these United Colonies are, and of Right ought to be Free and Independent States; that they are Absolved from all Allegiance to the British Crown, and that all political connection between them and the State of Great Britain, is and ought to be totally dissolved; and that as Free and Independent States, they have full Power to levy War, conclude Peace, contract Alliances, establish Commerce, and to do all other Acts and Things which Independent States may of right do. And for the support of this Declaration, with a firm reliance on the protection of divine Providence, we mutually pledge to each other our Lives, our Fortunes and our sacred Honor."""


In [4]:
import re

def text_cleaner(text):
    # lower case text
    newString = text.lower()
    newString = re.sub(r"'s\b","",newString)
    # remove punctuations
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    long_words=[]
    # remove short word
    for i in newString.split():
        if len(i)>=3:                  
            long_words.append(i)
    return (" ".join(long_words)).strip()

# preprocess the text
data_new = text_cleaner(data_text)
data_new

'the unanimous declaration the thirteen united states america when the course human events becomes necessary for one people dissolve the political bands which have connected them with another and assume among the powers the earth the separate and equal station which the laws nature and nature god entitle them decent respect the opinions mankind requires that they should declare the causes which impel them the separation hold these truths self evident that all men are created equal that they are endowed their creator with certain unalienable rights that among these are life liberty and the pursuit happiness that secure these rights governments are instituted among men deriving their just powers from the consent the governed that whenever any form government becomes destructive these ends the right the people alter abolish and institute new government laying its foundation such principles and organizing its powers such form them shall seem most likely effect their safety and happiness pr

## Creating sequences of text for model
1. Since we are building chracter level model so we'll so how to create sequences

In [33]:
## We will take in 30 characters as context 
def create_seq(text):
    length = 30
    dum_sequences = list()
    ori_sequences = list()
    for i in range(length, len(text)):
        # select sequence of tokens
        seq = text[i-length:i+1]
        seq2 = [char for char in seq]
        # store
        dum_sequences.append(seq)
        ori_sequences.append(seq2) 
    print('Total Sequences: %d' % len(sequences))
    return (dum_sequences , ori_sequences)

# create sequences   
dum_sequences , ori_sequences = create_seq(data_new)
dum_sequences[:10]

Total Sequences: 1


['the unanimous declaration the t',
 'he unanimous declaration the th',
 'e unanimous declaration the thi',
 ' unanimous declaration the thir',
 'unanimous declaration the thirt',
 'nanimous declaration the thirte',
 'animous declaration the thirtee',
 'nimous declaration the thirteen',
 'imous declaration the thirteen ',
 'mous declaration the thirteen u']

In [34]:
ori_sequences[:2]

[['t',
  'h',
  'e',
  ' ',
  'u',
  'n',
  'a',
  'n',
  'i',
  'm',
  'o',
  'u',
  's',
  ' ',
  'd',
  'e',
  'c',
  'l',
  'a',
  'r',
  'a',
  't',
  'i',
  'o',
  'n',
  ' ',
  't',
  'h',
  'e',
  ' ',
  't'],
 ['h',
  'e',
  ' ',
  'u',
  'n',
  'a',
  'n',
  'i',
  'm',
  'o',
  'u',
  's',
  ' ',
  'd',
  'e',
  'c',
  'l',
  'a',
  'r',
  'a',
  't',
  'i',
  'o',
  'n',
  ' ',
  't',
  'h',
  'e',
  ' ',
  't',
  'h']]

## Encoding sentences
As encoding can't be done on above character level by using funtions . So neeed to manually do it

In [39]:
## Method #1
## Mapping index for each character of total 26

chars = sorted(list(set(data_new)))
print(list(chars))

mapping = dict((c, i) for i, c in enumerate(chars))
print(list(mapping.values()))

def encoded_seq(seq):
    sequences = list()
    for line in seq:
        encoded_seq = [mapping[char] for char in line]
        sequences.append(encoded_seq)
    return sequences

sequences = encoded_seq(dum_sequences)
sequences[:3]

[' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]


[[20,
  8,
  5,
  0,
  21,
  14,
  1,
  14,
  9,
  13,
  15,
  21,
  19,
  0,
  4,
  5,
  3,
  12,
  1,
  18,
  1,
  20,
  9,
  15,
  14,
  0,
  20,
  8,
  5,
  0,
  20],
 [8,
  5,
  0,
  21,
  14,
  1,
  14,
  9,
  13,
  15,
  21,
  19,
  0,
  4,
  5,
  3,
  12,
  1,
  18,
  1,
  20,
  9,
  15,
  14,
  0,
  20,
  8,
  5,
  0,
  20,
  8],
 [5,
  0,
  21,
  14,
  1,
  14,
  9,
  13,
  15,
  21,
  19,
  0,
  4,
  5,
  3,
  12,
  1,
  18,
  1,
  20,
  9,
  15,
  14,
  0,
  20,
  8,
  5,
  0,
  20,
  8,
  9]]

In [41]:
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.tokenize import word_tokenize


In [42]:
## Method #2
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(ori_sequences)
sequences_2 = tokenizer_obj.texts_to_sequences(ori_sequences)
sequences_2[:3]

[[3,
  10,
  2,
  1,
  13,
  4,
  5,
  4,
  8,
  15,
  9,
  13,
  6,
  1,
  11,
  2,
  14,
  12,
  5,
  7,
  5,
  3,
  8,
  9,
  4,
  1,
  3,
  10,
  2,
  1,
  3],
 [10,
  2,
  1,
  13,
  4,
  5,
  4,
  8,
  15,
  9,
  13,
  6,
  1,
  11,
  2,
  14,
  12,
  5,
  7,
  5,
  3,
  8,
  9,
  4,
  1,
  3,
  10,
  2,
  1,
  3,
  10],
 [2,
  1,
  13,
  4,
  5,
  4,
  8,
  15,
  9,
  13,
  6,
  1,
  11,
  2,
  14,
  12,
  5,
  7,
  5,
  3,
  8,
  9,
  4,
  1,
  3,
  10,
  2,
  1,
  3,
  10,
  8]]

In [46]:
#sequences = np.array(sequences)
sequences[:,:-1] # Rows sari aur colunm -1 ( means ky 1 kam)

array([[20,  8,  5, ...,  8,  5,  0],
       [ 8,  5,  0, ...,  5,  0, 20],
       [ 5,  0, 21, ...,  0, 20,  8],
       ...,
       [15, 21, 18, ...,  0,  8, 15],
       [21, 18,  0, ...,  8, 15, 14],
       [18,  0,  6, ..., 15, 14, 15]])

In [59]:
from sklearn.model_selection import train_test_split

# vocabulary size
vocab = len(mapping)
sequences = np.array(sequences)
# create X and y
X, y = sequences[:,:-1], sequences[:,-1]
print(y)
y = to_categorical(y, num_classes=vocab)
print(y.shape , len(y))
y


[20  8  9 ... 14 15 18]
(7052, 27) 7052


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [60]:
# create train and validation sets
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

print('Train shape:', X_tr.shape, 'Val shape:', X_val.shape)

Train shape: (6346, 30) Val shape: (706, 30)


In [69]:
# define model
model = Sequential()
model.add(Embedding(vocab, 50, input_length=30, trainable=True))
model.add(Bidirectional(LSTM(150, recurrent_dropout=0.1, dropout=0.1)))
model.add(Dense(vocab, activation='softmax'))
print(model.summary())

# compile the model
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')
# fit the model
model.fit(X_tr, y_tr, epochs=50, verbose=2, validation_data=(X_val, y_val))

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 30, 50)            1350      
_________________________________________________________________
bidirectional_3 (Bidirection (None, 300)               241200    
_________________________________________________________________
dense_5 (Dense)              (None, 27)                8127      
Total params: 250,677
Trainable params: 250,677
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
199/199 - 46s - loss: 2.8859 - acc: 0.1491 - val_loss: 2.7979 - val_acc: 0.1983
Epoch 2/50
199/199 - 32s - loss: 2.5138 - acc: 0.2684 - val_loss: 2.3414 - val_acc: 0.3088
Epoch 3/50
199/199 - 36s - loss: 2.2398 - acc: 0.3375 - val_loss: 2.2249 - val_acc: 0.3442
Epoch 4/50
199/199 - 35s - loss: 2.1251 - acc: 0.3613 - val_loss: 2.1258 - val_acc: 0.3782
Epoch 5/5

In [70]:
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
	in_text = seed_text
	# generate a fixed number of characters
	for _ in range(n_chars):
		# encode the characters as integers
		encoded = [mapping[char] for char in in_text]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict character
		yhat = model.predict_classes(encoded, verbose=0)
		# reverse map integer to character
		out_char = ''
		for char, index in mapping.items():
			if index == yhat:
				out_char = char
				break
		# append to input
		in_text += char
	return in_text

In [72]:
predicted = generate_seq(model , mapping , 30 , 'the unanimous declaration the ' , 1)
print(predicted)

the unanimous declaration the t


In [73]:
predicted = generate_seq(model , mapping , 30 , 'e unanimous declaration the th' , 1)
print(predicted)

e unanimous declaration the thi


In [74]:
predicted = generate_seq(model , mapping , 30 , 'unanimous declaration the thir' , 1)
print(predicted)

unanimous declaration the thirt


In [75]:
predicted = generate_seq(model , mapping , 30 , 'nimous declaration the thirtee' , 1)
print(predicted)

nimous declaration the thirteen


## Waooo it works !!!